# Sentiment Analysis Using Naive Bayes

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("E:/datasets/sentiment_analysis_movie_dataset/IMDB Dataset.csv")

In [3]:
movies.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
movies['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

# Steps 
- Text Preprocessing/Cleaning
- Text vectorization => Bag of words/ TFIDF/ WORD2VEC/ Embeddings
- Data Modelling => Naive Bayes

# Text Cleaning Process
- 1. Remove HTML tags
- 2. Remove special characters
- 3. Converting every text into lower case
- 4. Removing stopwords
- 5. Stemming

In [5]:
movies.shape

(50000, 2)

In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [7]:
movies['sentiment'].replace({'positive':1, 'negative':0}, inplace=True)

In [8]:
movies.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


# 1. Removing HTML Tags

In [9]:
import re
clean = re.compile("<.*?>")
re.sub(clean, '', movies.iloc[1].review)

'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well done.'

In [10]:
movies.iloc[1].review

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [11]:
# function for clean the html tags 
def clean_html(text):
    clean = re.compile("<.*?>")
    return re.sub(clean, '', text)

In [12]:
movies['review'] = movies['review'].apply(clean_html)

# 2. Convert into lower text

In [13]:
# convert into lower text
def lower_text(text):
    return text.lower()

In [14]:
movies['review'] = movies['review'].apply(lower_text)

In [15]:
movies['review'][0]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

# 3. Remove the special characters

In [16]:
def remove_special(text):
    x = ''
    for i in text:
        if i.isalnum():
            x = x + i
        else:
            x = x + ' '
    return x

In [17]:
remove_special("one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. forget pretty pictures painted for mainstream audiences, forget charm, forget romance...oz doesn't mess around. the first episode i ever saw struck me as so nasty it was surreal, i couldn't say i was ready for it, but as i watched more, i developed a taste for oz, and got accustomed to the high levels of graphic violence. not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) watching oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.")

'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [18]:
movies['review'] = movies['review'].apply(remove_special)

In [19]:
movies.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there s a family where a little boy ...,0
4,petter mattei s love in the time of money is...,1


# 4. Remove the stop words

In [20]:
import nltk

In [21]:
from nltk.corpus import stopwords

In [22]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [23]:
def remove_stopwords(text):
    x = []
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y = x[:]
    x.clear()
    return y

In [24]:
movies['review'] = movies['review'].apply(remove_stopwords)

In [25]:
movies.head()

,review,sentiment
0,"[one, reviewers, mentioned, watching, 1, oz, e...",1
1,"[wonderful, little, production, filming, techn...",1
2,"[thought, wonderful, way, spend, time, hot, su...",1
3,"[basically, family, little, boy, jake, thinks,...",0
4,"[petter, mattei, love, time, money, visually, ...",1


# Perform Stemming 

In [26]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [28]:
y = []
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z = y[:]
    y.clear()
    return z

In [32]:
stem_words(['I', 'loved', 'loving', 'it'])

['i', 'love', 'love', 'it']

In [33]:
movies['review'] = movies['review'].apply(stem_words)

In [34]:
movies.head()

,review,sentiment
0,"[one, review, mention, watch, 1, oz, episod, h...",1
1,"[wonder, littl, product, film, techniqu, unass...",1
2,"[thought, wonder, way, spend, time, hot, summe...",1
3,"[basic, famili, littl, boy, jake, think, zombi...",0
4,"[petter, mattei, love, time, money, visual, st...",1


In [35]:
# join back
def join_back(list_input):
    return " ".join(list_input)

In [36]:
movies['review'] = movies['review'].apply(join_back)

In [37]:
movies['review']

0        one review mention watch 1 oz episod hook righ...
1        wonder littl product film techniqu unassum old...
2        thought wonder way spend time hot summer weeke...
3        basic famili littl boy jake think zombi closet...
4        petter mattei love time money visual stun film...
                               ...                        
49995    thought movi right good job creativ origin fir...
49996    bad plot bad dialogu bad act idiot direct anno...
49997    cathol taught parochi elementari school nun ta...
49998    go disagre previou comment side maltin one sec...
49999    one expect star trek movi high art fan expect ...
Name: review, Length: 50000, dtype: object

In [38]:
X = movies.iloc[:,0:1].values

In [39]:
X.shape

(50000, 1)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)

In [41]:
X = cv.fit_transform(movies['review']).toarray()

In [42]:
X.shape

(50000, 5000)

In [43]:
X[0].mean()

0.0286

In [44]:
y = movies.iloc[:, -1].values

In [45]:
y.shape

(50000,)

# Data Modelling using Naive bayes 

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [47]:
X_train.shape

(40000, 5000)

In [48]:
X_test.shape

(10000, 5000)

In [49]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [50]:
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()

In [51]:
clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train, y_train)

BernoulliNB()

In [52]:
y_pred1 = clf1.predict(X_test)
y_pred2 = clf2.predict(X_test)
y_pred3 = clf3.predict(X_test)

In [53]:
y_test.shape

(10000,)

In [54]:
y_pred1.shape

(10000,)

# Metrics

In [55]:
from sklearn.metrics import accuracy_score

In [56]:
print("Gaussian", accuracy_score(y_test, y_pred1))
print("Multinomial", accuracy_score(y_test, y_pred2))
print("Bernoulli", accuracy_score(y_test, y_pred3))

Gaussian 0.7284
Multinomial 0.8466
Bernoulli 0.8514


In [57]:
from sklearn.metrics import classification_report, confusion_matrix

In [58]:
report1 = classification_report(y_test,y_pred1)
report2 = classification_report(y_test,y_pred2)
report3 = classification_report(y_test,y_pred3)

In [59]:
print(report1)
print("*"*55)
print(report2)
print("*"*55)
print(report3)
print("*"*55)

              precision    recall  f1-score   support

           0       0.68      0.86      0.76      4989
           1       0.81      0.60      0.69      5011

    accuracy                           0.73     10000
   macro avg       0.75      0.73      0.72     10000
weighted avg       0.75      0.73      0.72     10000

*******************************************************
              precision    recall  f1-score   support

           0       0.84      0.85      0.85      4989
           1       0.85      0.84      0.85      5011

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

*******************************************************
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      4989
           1       0.85      0.85      0.85      5011

    accuracy                           0.85     10000
   macro avg    

In [60]:
cmt1 = confusion_matrix(y_test, y_pred1)
cmt2 = confusion_matrix(y_test, y_pred2)
cmt3 = confusion_matrix(y_test, y_pred3)

In [61]:
print(cmt1)
print("*"*15)
print(cmt2)
print("*"*15)
print(cmt3)
print("*"*15)

[[4292  697]
 [2019 2992]]
***************
[[4237  752]
 [ 782 4229]]
***************
[[4249  740]
 [ 746 4265]]
***************
